## Deep Dictionary Learning implementation

Julia implementation of the paper avaliable at: 

https://arxiv.org/pdf/1602.00203.pdf

In [81]:
using Pkg
Pkg.add("PyCall")
Pkg.add("DataStructures")
Pkg.add("SparseArrays")
Pkg.add("ProgressBars")
Pkg.add("DelimitedFiles")
Pkg.add("Plots")
Pkg.add("Images")
using DataStructures
using SparseArrays
using LinearAlgebra
using ProgressBars
using PyCall
using Plots

 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]
 Res

In [82]:
function init_dictionary(n::Int, K::Int)
    """
    Initialize the dictionary.
    Args: 
        n: dimension of input signal
        k = number of atoms in the dictionary
    """
    # D must be a full-rank matrix
    D = rand(n, K)
    while rank(D) != min(n, K)
        D = rand(n, K)
    end

    @inbounds for k in 1:K
        D[:, k] ./= norm(@view(D[:, k]))
    end
    return D
end

init_dictionary (generic function with 1 method)

In [215]:
function dense_dict_learning(x,nb_iters,n_atoms)
    """
    Computes a dense dictionary learned layer with the correspondent learned representation.
    
    Args:
    x - features to learn
    nb_iters: number of iterations for the algorithm
    n_atoms - number of atoms in the dictionary
    
    returns: 
    Dictionary (D) and dense coefficients (Z)
    
    """
    
    D = init_dictionary(size(x,1),n_atoms)
    Z = zeros(n_atoms,size(x,2))
    
    Z = inv((transpose(D)*D))*transpose(D)*x
    D = x*transpose(Z)*inv(Z*transpose(Z))

    return D,Z
    
end

dense_dict_learning (generic function with 1 method)

In [216]:
#Define singular value soft thresholding (SVST)
SVST = (X,beta) -> begin
    U,s,V = svd(X)
    sthresh = max.(s .- beta,0)
    return U * Diagonal(sthresh) * V'
end;

In [217]:
function ISTA(y)
    # Define cost function for optimization problem
    Omega = y .!= 0
    nucnorm = (X) -> sum(svdvals(X))
    costfun = (X,beta) -> 0.5 * norm(X[Omega]-y[Omega])^2 + beta * nucnorm(X);
    # Apply ISTA (Iterative Soft-Thresholding Algorithm)
    niter = 400
    X = copy(y)
    Xold = copy(X)
    beta = 0.01 
    cost_ista = zeros(niter+1)
    cost_ista[1] = costfun(X,beta)
    for k in ProgressBar(1:niter)
        X[Omega] = y[Omega]
        X = SVST(X,beta)
        cost_ista[k+1] = costfun(X,beta)
    end
    return X
end

ISTA (generic function with 1 method)

Learning the sucessive dictionaries: _(Deep)_

In [218]:
function learn(X)
    nb_iters = 10
    n_atoms = 100
    D1,Z1 = dense_dict_learning(X,nb_iters,300)
    D2,Z2 = dense_dict_learning(Z1,nb_iters,15)
    Z3 = ISTA(Z2)
    D3 = Z2*transpose(Z3)*inv(Z3*transpose(Z3))
    
    return D1,D2,D3,Z3
end

learn (generic function with 1 method)

In [219]:
datasets = pyimport("sklearn.datasets")
digits = datasets.load_digits()
Y = digits["data"];

In [220]:
D1,D2,D3,Z3 = learn(Y);

100.0%┣█████████████████████████████████████████████████████████┫ 400/400 [00:00<00:00, 1668.2 it/s]


In [222]:
using DelimitedFiles

writedlm("dlm_files/dictionary_D1.dlm", D1)
writedlm("dlm_files/dictionary_D2.dlm", D2)
writedlm("dlm_files/dictionary_D3.dlm", D3)
writedlm("dlm_files/Z3.dlm", Z3)